## Project goal -
Using the Telco data:
- determine the drivers of customer churn
- make recommendations to decrease churn
- present findings to stakeholders

In [11]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import numpy as np

from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns
import acquire
import prepare

#### Acquire data

In [17]:
df = acquire.get_telco_data()
df.head()

,Unnamed: 0,contract_type_id,payment_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,1,2,1,0003-MKNFE,Male,0,No,No,9,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month
2,2,1,1,2,0004-TLHLJ,Male,0,No,No,4,...,No,No,No,Yes,73.9,280.85,Yes,Fiber optic,Electronic check,Month-to-month
3,3,1,1,2,0011-IGKFF,Male,1,Yes,No,13,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Fiber optic,Electronic check,Month-to-month
4,4,1,2,2,0013-EXCHZ,Female,1,Yes,No,3,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Fiber optic,Mailed check,Month-to-month


#### Prepare data

Use prep_telco function to begin cleaning data. This will drop columns that wouldn't be appropriate for the model and create dummy variables to change object types to numeric.

In [18]:
df = df.drop(['Unnamed: 0', 'customer_id'], axis=1)
dummy = pd.get_dummies(df[['gender', 'partner', 'dependents', 'phone_service', 'multiple_lines',
                  'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing',
                  'churn', 'internet_service_type', 'payment_type', 'contract_type']], #prefix= (['encoded', 'encoded', 'encoded', 'encoded', 'encoded', 'encoded', 'encoded', 'encoded', 'encoded', 'encoded', 'encoded', 'encoded', 'encoded']),
              dummy_na=False, drop_first=[True, True, True, True, True, True, True, True, True, True, True, True, True])
   

In [19]:
dummy.columns

Index(['gender_Male', 'partner_Yes', 'dependents_Yes', 'phone_service_Yes',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'tech_support_No internet service', 'tech_support_Yes',
       'streaming_tv_No internet service', 'streaming_tv_Yes',
       'streaming_movies_No internet service', 'streaming_movies_Yes',
       'paperless_billing_Yes', 'churn_Yes',
       'internet_service_type_Fiber optic', 'internet_service_type_None',
       'payment_type_Credit card (automatic)', 'payment_type_Electronic check',
       'payment_type_Mailed check', 'contract_type_One year',
       'contract_type_Two year'],
      dtype='object')

rename all of the dummy variable columns to start with e_ for "encoded_" which will make it easier to differentiate features that are represented as both discrete (objects, original data) and numeric (dummy, 1 = true)

In [20]:
dummy.rename(columns = {'gender_Male': 'e_gender_male', 'partner_Yes':'e_partner', 'dependents_Yes':'e_dependents',
                     'phone_service_Yes': 'e_phoneserv','multiple_lines_No phone service': 'e_multi_no_phone', 'multiple_lines_Yes': 'e_multi',
       'tech_support_No internet service': 'e_tech_no_int', 'tech_support_Yes': 'e_tech',
       'streaming_tv_No internet service': 'e_tv_no_int', 'streaming_tv_Yes': 'e_tv',
       'streaming_movies_No internet service': 'e_movies_no_int', 'streaming_movies_Yes': 'e_movies',
       'paperless_billing_Yes': 'e_paperless_bill', 'churn_Yes': 'e_churn',
       'internet_service_type_Fiber optic' : 'e_fiber_optic', 'internet_service_type_None': 'e_no_int',
       'payment_type_Credit card (automatic)': 'e_payment_cc', 'payment_type_Electronic check': 'e_payment_check_el',
       'payment_type_Mailed check': 'e_check_mail', 'contract_type_One year': 'e_contract_oneyr',
       'contract_type_Two year': 'e_contract_twoyr'}, inplace = True)

In [21]:
dummy.columns

Index(['e_gender_male', 'e_partner', 'e_dependents', 'e_phoneserv',
       'e_multi_no_phone', 'e_multi', 'e_tech_no_int', 'e_tech', 'e_tv_no_int',
       'e_tv', 'e_movies_no_int', 'e_movies', 'e_paperless_bill', 'e_churn',
       'e_fiber_optic', 'e_no_int', 'e_payment_cc', 'e_payment_check_el',
       'e_check_mail', 'e_contract_oneyr', 'e_contract_twoyr'],
      dtype='object')

concatenate new dummy dataframe and original dataframe to work with one large dataframe 

In [22]:
df = pd.concat([dummy, df], axis=1)

In [23]:
df.head()

,e_gender_male,e_partner,e_dependents,e_phoneserv,e_multi_no_phone,e_multi,e_tech_no_int,e_tech,e_tv_no_int,e_tv,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,0,1,1,1,0,0,0,1,0,1,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,0,0,1,0,1,0,0,0,0,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month
2,1,0,0,1,0,0,0,0,0,0,...,No,No,No,Yes,73.9,280.85,Yes,Fiber optic,Electronic check,Month-to-month
3,1,1,0,1,0,0,0,0,0,1,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Fiber optic,Electronic check,Month-to-month
4,0,1,0,1,0,0,0,1,0,1,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Fiber optic,Mailed check,Month-to-month


In [25]:
df.isna().sum()

e_gender_male               0
e_partner                   0
e_dependents                0
e_phoneserv                 0
e_multi_no_phone            0
e_multi                     0
e_tech_no_int               0
e_tech                      0
e_tv_no_int                 0
e_tv                        0
e_movies_no_int             0
e_movies                    0
e_paperless_bill            0
e_churn                     0
e_fiber_optic               0
e_no_int                    0
e_payment_cc                0
e_payment_check_el          0
e_check_mail                0
e_contract_oneyr            0
e_contract_twoyr            0
contract_type_id            0
payment_type_id             0
internet_service_type_id    0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_pro

In [30]:
df.dtypes

e_gender_male                 uint8
e_partner                     uint8
e_dependents                  uint8
e_phoneserv                   uint8
e_multi_no_phone              uint8
e_multi                       uint8
e_tech_no_int                 uint8
e_tech                        uint8
e_tv_no_int                   uint8
e_tv                          uint8
e_movies_no_int               uint8
e_movies                      uint8
e_paperless_bill              uint8
e_churn                       uint8
e_fiber_optic                 uint8
e_no_int                      uint8
e_payment_cc                  uint8
e_payment_check_el            uint8
e_check_mail                  uint8
e_contract_oneyr              uint8
e_contract_twoyr              uint8
contract_type_id              int64
payment_type_id               int64
internet_service_type_id      int64
gender                       object
senior_citizen                int64
partner                      object
dependents                  

total_charges column to be converted to numeric

In [31]:
df['total_charges'] = pd.to_numeric(df['total_charges'], errors='coerce')
df.total_charges

0        593.30
1        542.40
2        280.85
3       1237.85
4        267.40
         ...   
7038     742.90
7039    1873.70
7040      92.75
7041    4627.65
7042    3707.60
Name: total_charges, Length: 7043, dtype: float64

In [32]:
df.isna().sum()

e_gender_male                0
e_partner                    0
e_dependents                 0
e_phoneserv                  0
e_multi_no_phone             0
e_multi                      0
e_tech_no_int                0
e_tech                       0
e_tv_no_int                  0
e_tv                         0
e_movies_no_int              0
e_movies                     0
e_paperless_bill             0
e_churn                      0
e_fiber_optic                0
e_no_int                     0
e_payment_cc                 0
e_payment_check_el           0
e_check_mail                 0
e_contract_oneyr             0
e_contract_twoyr             0
contract_type_id             0
payment_type_id              0
internet_service_type_id     0
gender                       0
senior_citizen               0
partner                      0
dependents                   0
tenure                       0
phone_service                0
multiple_lines               0
online_security              0
online_b

In [40]:
df.groupby(df.total_charges).sum()

,e_gender_male,e_partner,e_dependents,e_phoneserv,e_multi_no_phone,e_multi,e_tech_no_int,e_tech,e_tv_no_int,e_tv,...,e_payment_check_el,e_check_mail,e_contract_oneyr,e_contract_twoyr,contract_type_id,payment_type_id,internet_service_type_id,senior_citizen,tenure,monthly_charges
total_charges,,,,,,,,,,,,,,,,,,,,,
18.80,1,1,1,1,0,0,1,0,1,0,...,0,1,1,0,2,2,3,0,1,18.80
18.85,1,0,0,2,0,0,2,0,2,0,...,1,1,0,0,2,3,6,0,2,37.70
18.90,1,0,0,1,0,0,1,0,1,0,...,0,1,0,0,1,2,3,0,1,18.90
19.00,0,1,1,1,0,0,1,0,1,0,...,0,1,0,0,1,2,3,0,1,19.00
19.05,1,0,0,1,0,0,1,0,1,0,...,0,1,0,0,1,2,3,0,1,19.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8564.75,1,1,0,1,0,1,0,1,0,1,...,1,0,0,1,3,1,2,0,71,116.25
8594.40,1,1,0,1,0,1,0,1,0,1,...,0,0,0,1,3,4,2,0,72,116.95
8670.10,0,1,1,1,0,1,0,1,0,1,...,0,0,0,1,3,4,2,0,72,117.50


In [49]:
df[df.total_charges.isna()].tenure

945     0
1731    0
1906    0
2025    0
2176    0
2250    0
2855    0
3052    0
3118    0
4054    0
5433    0
Name: tenure, dtype: int64

After changing data type of total_charges to float, there are 11 null values. All of these null values are in rows with new customers (represented by 0 tenure). Brand new customers would not hold any significance to determine churn so dropping these rows.

In [50]:
df.dropna(subset=['total_charges'], inplace=True)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   e_gender_male             7032 non-null   uint8  
 1   e_partner                 7032 non-null   uint8  
 2   e_dependents              7032 non-null   uint8  
 3   e_phoneserv               7032 non-null   uint8  
 4   e_multi_no_phone          7032 non-null   uint8  
 5   e_multi                   7032 non-null   uint8  
 6   e_tech_no_int             7032 non-null   uint8  
 7   e_tech                    7032 non-null   uint8  
 8   e_tv_no_int               7032 non-null   uint8  
 9   e_tv                      7032 non-null   uint8  
 10  e_movies_no_int           7032 non-null   uint8  
 11  e_movies                  7032 non-null   uint8  
 12  e_paperless_bill          7032 non-null   uint8  
 13  e_churn                   7032 non-null   uint8  
 14  e_fiber_

use train_validate_test_split function to split the data into three dataframes:

train- .70 * .80= 56% of the original dataset
validate- .30 * .80= 24% of the original dataset
test- 20% of the original dataset

Stratify the target variable ('e_churn') to ensure subsets proportionally represent the original dataframe

Note: 'e_churn' -
1 = has churned
0 = has no churned

In [52]:
from sklearn.linear_model import LogisticRegression

In [53]:
def train_validate_test_split(df, target, seed=123):
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed,
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3,
                                       random_state=seed,
                                       stratify=train_validate[target])
        
    return train, validate, test

In [54]:
train, validate, test = train_validate_test_split(df, target='e_churn', seed=123)
train.shape, validate.shape, test.shape

((3937, 44), (1688, 44), (1407, 44))

In [55]:
train.e_churn.value_counts()

0    2891
1    1046
Name: e_churn, dtype: int64

determine baseline accuracy

In [77]:
baseline_accuracy = ((train.e_churn == 0).sum() / len(train.e_churn))
baseline_accuracy
# 73.4% of customers do not churn on baseline accuracy

0.7343154686309372

split train, validate, and test data into X and y subsets to prep for modeling. <br> 
note - discrete columns and columns with duplicate data are dropped

### Logistic Regression Model

In [79]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3937 entries, 5919 to 4192
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   e_gender_male             3937 non-null   uint8  
 1   e_partner                 3937 non-null   uint8  
 2   e_dependents              3937 non-null   uint8  
 3   e_phoneserv               3937 non-null   uint8  
 4   e_multi_no_phone          3937 non-null   uint8  
 5   e_multi                   3937 non-null   uint8  
 6   e_tech_no_int             3937 non-null   uint8  
 7   e_tech                    3937 non-null   uint8  
 8   e_tv_no_int               3937 non-null   uint8  
 9   e_tv                      3937 non-null   uint8  
 10  e_movies_no_int           3937 non-null   uint8  
 11  e_movies                  3937 non-null   uint8  
 12  e_paperless_bill          3937 non-null   uint8  
 13  e_churn                   3937 non-null   uint8  
 14  e_fib

In [136]:
X_train = train.drop(columns=['contract_type_id', 'payment_type_id', 'internet_service_type_id', 'gender',
                             'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security',
                             'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                             'paperless_billing', 'churn', 'internet_service_type', 'payment_type', 'contract_type', 'e_churn'])
y_train = train.e_churn

X_validate = validate.drop(columns=['contract_type_id', 'payment_type_id', 'internet_service_type_id', 'gender',
                             'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security',
                             'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                             'paperless_billing', 'churn', 'internet_service_type', 'payment_type', 'contract_type', 'e_churn'])
y_validate = validate.e_churn

X_test = test.drop(columns=['contract_type_id', 'payment_type_id', 'internet_service_type_id', 'gender',
                             'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security',
                             'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                             'paperless_billing', 'churn', 'internet_service_type', 'payment_type', 'contract_type', 'e_churn'])
y_test = test.e_churn

In [147]:
logit = LogisticRegression(C=.1, class_weight={0: 1, 1: 99}, random_state=123, intercept_scaling=1, solver='lbfgs')

In [148]:
logit.fit(X_train, y_train)
LogisticRegression(C=.1, class_weight={0: 1, 1: 99}, random_state=123)

LogisticRegression(C=0.1, class_weight={0: 1, 1: 99}, random_state=123)

In [149]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

Coefficient: 
 [[ 1.26675252e-01 -1.15287272e-01  4.88697046e-03 -8.80679838e-02
   1.22968128e+00  2.67392751e-02  1.78672429e-01 -5.99957300e-01
   1.78672429e-01 -3.89310827e-01  1.78672429e-01 -3.13621389e-01
   4.27392396e-01 -1.72822767e-01  1.78672429e-01  3.08780071e-01
   6.41983174e-01 -1.53508050e-01 -4.18756460e-01 -1.06788118e+00
   6.52532003e-01 -5.11122191e-02  5.21014282e-02  7.66615824e-05]]
Intercept: 
 [1.16406384]


In [150]:
y_pred = logit.predict(X_train)
y_pred_proba = logit.predict_proba(X_train)

y_pred, y_pred_proba

(array([1, 1, 1, ..., 1, 1, 1], dtype=uint8),
 array([[0.13128094, 0.86871906],
        [0.06512046, 0.93487954],
        [0.08342148, 0.91657852],
        ...,
        [0.2347658 , 0.7652342 ],
        [0.00620887, 0.99379113],
        [0.06241303, 0.93758697]]))

In [151]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set: 0.34
